In [1]:
import requests
from tqdm import tqdm_notebook as tqdm
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
Base = "http://race.sp.netkeiba.com/?pid=race_result&race_id="
dst = ''
df_col = ['year', 'date', 'field', 'race', 'race_name'
          , 'course', 'head_count', 'rank', 'horse_name'
          , 'gender', 'age', 'trainerA', 'trainerB', 'weight', 'c_weight', 'jackie', 'j_weight'
          , 'odds','popu']

In [3]:
def numStr(num):
    if num >= 10:
        return str(num)
    else:
        return '0' + str(num)

In [4]:
year = 2008
i = 1
j = 1
k = 1
l = 1

In [5]:
url = Base + str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l)

In [7]:
start = time.time()
html = requests.get(url)
print(time.time()-start)

0.10069680213928223


In [9]:
html.encoding = 'EUC-JP'
start = time.time()
soup = BeautifulSoup(html.text, 'html.parser')
print(time.time()-start)

1.109039068222046


In [11]:
start = time.time()
CommonYear = year
CommonDate = soup.find_all('div', attrs={'class', 'Change_Btn Day'})[0].string.strip()
CommonField= soup.find_all('div', attrs={'class', 'Change_Btn Course'})[0].string.strip()
CommonRace = soup.find_all('div', attrs={'Race_Num'})[0].span.string
CommonRname= soup.find_all('dt', attrs={'class', 'Race_Name'})[0].contents[0].strip()
CommonCourse= soup.find_all('dd', attrs={'Race_Data'})[0].span.string
CommonHcount= soup.find_all('dd', attrs={'class', 'Race_Data'})[0].contents[3].split()[1]
print(time.time()-start)

0.15761232376098633


In [23]:
start = time.time()
df = pd.DataFrame(columns=df_col)
m = 1
dst = pd.Series(index=df_col)
dst['year'] = CommonYear
dst['date'] = CommonDate
dst['field']= CommonField
dst['race'] = CommonRace
dst['race_name'] = CommonRname
dst['course'] = CommonCourse
dst['head_count'] = CommonHcount
dst['rank'] = soup.find_all('div', attrs='Rank')[m].contents[0]
dst['horse_name'] = soup.find_all('dt', attrs=['class', 'Horse_Name'])[m].a.string
detailL = soup.find_all('span', attrs=['class', 'Detail_Left'])[m]
dst['gender'] = list(detailL.contents[0].split()[0])[0]
dst['age'] = list(detailL.contents[0].split()[0])[1]
dst['trainerA'] = detailL.span.string.split('･')[0]
dst['trainerB'] = detailL.span.string.split('･')[1]
if len(detailL.contents[0].split())>=2:
    dst['weight'] = detailL.contents[0].split()[1].split('(')[0]
    if len(detailL.contents[0].split()[1].split('('))>=2:
        dst['c_weight'] = detailL.contents[0].split()[1].split('(')[1].strip(')')
        detailR = soup.find_all('span', attrs=['class', 'Detail_Right'])[m].contents
        if  "\n" in detailR or "\n▲" in detailR or '\n☆' in detailR:
            detailR.pop(0)
        dst['jackie'] = detailR[0].string.strip()
        dst['j_weight'] = detailR[2].strip().replace('(', '').replace(')', '')                    
        Odds = soup.find_all('td', attrs=['class', 'Odds'])[m].contents[1]
        if Odds.dt.string is not None:
            dst['odds'] = Odds.dt.string.strip('倍')
dst.name = str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l) + numStr(m)
df = df.append(dst)
print(time.time()-start)

0.1655569076538086


In [24]:
df

,year,date,field,race,race_name,course,head_count,rank,horse_name,gender,age,trainerA,trainerB,weight,c_weight,jackie,j_weight,odds,popu
20080101010101,2008.0,8/16(土),札幌,1R,２歳未勝利,芝1500m,14頭,2,アイディンパワー,牡,2,栗東,本田,460,0,藤田,54.0,5.2,NaN


In [25]:
for year in tqdm(range(2015, 2019)):
    df = pd.DataFrame()
    for i in tqdm(range(1, 11)):
        for j in tqdm(range(1, 11)):
            for k in range(1, 11):
                for l in range(1, 13):
                    # urlでぶっこ抜く
                    url = Base + str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l)
                    #time.sleep(1)
                    html = requests.get(url)
                    html.encoding = 'EUC-JP'
                    
                    # scraping
                    soup = BeautifulSoup(html.text, 'html.parser')
                    # ページがあるかの判定
                    if soup.find_all('div', attrs={'class', 'Result_Guide'})!=[]:
                        time.sleep(1)
                        break
                    else:
                        #共通部分を抜き出す
                        CommonYear = year
                        CommonDate = soup.find_all('div', attrs={'class', 'Change_Btn Day'})[0].string.strip()
                        CommonField= soup.find_all('div', attrs={'class', 'Change_Btn Course'})[0].string.strip()
                        CommonRace = soup.find_all('div', attrs={'Race_Num'})[0].span.string
                        CommonRname= soup.find_all('dt', attrs={'class', 'Race_Name'})[0].contents[0].strip()
                        CommonCourse= soup.find_all('dd', attrs={'Race_Data'})[0].span.string
                        CommonHcount= soup.find_all('dd', attrs={'class', 'Race_Data'})[0].contents[3].split()[1]
                        
                        for m in range(len(soup.find_all('div', attrs='Rank'))):
                            dst = pd.Series(index=df_col)
                            try:
                                dst['year'] = CommonYear
                                dst['date'] = CommonDate
                                dst['field']= CommonField
                                dst['race'] = CommonRace
                                dst['race_name'] = CommonRname
                                dst['course'] = CommonCourse
                                dst['head_count'] = CommonHcount
                                dst['rank'] = soup.find_all('div', attrs='Rank')[m].contents[0]
                                dst['horse_name'] = soup.find_all('dt', attrs=['class', 'Horse_Name'])[m].a.string
                                detailL = soup.find_all('span', attrs=['class', 'Detail_Left'])[m]
                                dst['gender'] = list(detailL.contents[0].split()[0])[0]
                                dst['age'] = list(detailL.contents[0].split()[0])[1]
                                dst['trainerA'] = detailL.span.string.split('･')[0]
                                dst['trainerB'] = detailL.span.string.split('･')[1]
                                if len(detailL.contents[0].split())>=2:
                                    dst['weight'] = detailL.contents[0].split()[1].split('(')[0]
                                    if len(detailL.contents[0].split()[1].split('('))>=2:
                                        dst['c_weight'] = detailL.contents[0].split()[1].split('(')[1].strip(')')
                                detailR = soup.find_all('span', attrs=['class', 'Detail_Right'])[m].contents
                                if  "\n" in detailR or "\n▲" in detailR or '\n☆' in detailR:
                                    detailR.pop(0)
                                dst['jackie'] = detailR[0].string.strip()
                                dst['j_weight'] = detailR[2].strip().replace('(', '').replace(')', '')
                                Odds = soup.find_all('td', attrs=['class', 'Odds'])[m].contents[1]
                                if Odds.dt.string is not None:
                                    dst['odds'] = Odds.dt.string.strip('倍')
                            except:
                                time.sleep(0.01)
                                break
                            dst.name = str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l) + numStr(m)
                            
                            df = df.append(dst)
    df.to_csv('keiba' + str(year) + '.csv', encoding='shift-jis')

KeyboardInterrupt: 

In [ ]:
year = 2017
for i in tqdm(range(1, 11)):
    for j in tqdm(range(1, 11)):
        for k in range(1, 11):
            for l in range(1, 13):
                # urlでぶっこ抜く
                url = Base + str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l)
                #time.sleep(1)
                html = requests.get(url)
                html.encoding = 'EUC-JP'
                    
                # scraping
                soup = BeautifulSoup(html.text, 'html.parser')
                # ページがあるかの判定
                if soup.find_all('div', attrs={'class', 'Result_Guide'})!=[]:
                    time.sleep(1)
                    break
                else:
                    #共通部分を抜き出す
                    CommonYear = year
                    CommonDate = soup.find_all('div', attrs={'class', 'Change_Btn Day'})[0].string.strip()
                    CommonField= soup.find_all('div', attrs={'class', 'Change_Btn Course'})[0].string.strip()
                    CommonRace = soup.find_all('div', attrs={'Race_Num'})[0].span.string
                    CommonRname= soup.find_all('dt', attrs={'class', 'Race_Name'})[0].contents[0].strip()
                    CommonCourse= soup.find_all('dd', attrs={'Race_Data'})[0].span.string
                    CommonHcount= soup.find_all('dd', attrs={'class', 'Race_Data'})[0].contents[3].split()[1]
                        
                    for m in range(len(soup.find_all('div', attrs='Rank'))):
                        dst = pd.Series(index=df_col)
                        try:
                            dst['year'] = CommonYear
                            dst['date'] = CommonDate
                            dst['field']= CommonField
                            dst['race'] = CommonRace
                            dst['race_name'] = CommonRname
                            dst['course'] = CommonCourse
                            dst['head_count'] = CommonHcount
                            dst['rank'] = soup.find_all('div', attrs='Rank')[m].contents[0]
                            dst['horse_name'] = soup.find_all('dt', attrs=['class', 'Horse_Name'])[m].a.string
                            detailL = soup.find_all('span', attrs=['class', 'Detail_Left'])[m]
                            dst['gender'] = list(detailL.contents[0].split()[0])[0]
                            dst['age'] = list(detailL.contents[0].split()[0])[1]
                            dst['trainerA'] = detailL.span.string.split('･')[0]
                            dst['trainerB'] = detailL.span.string.split('･')[1]
                            if len(detailL.contents[0].split())>=2:
                                dst['weight'] = detailL.contents[0].split()[1].split('(')[0]
                                if len(detailL.contents[0].split()[1].split('('))>=2:
                                    dst['c_weight'] = detailL.contents[0].split()[1].split('(')[1].strip(')')
                            detailR = soup.find_all('span', attrs=['class', 'Detail_Right'])[m].contents
                            if  "\n" in detailR or "\n▲" in detailR or '\n☆' in detailR:
                                detailR.pop(0)
                            dst['jackie'] = detailR[0].string.strip()
                            dst['j_weight'] = detailR[2].strip().replace('(', '').replace(')', '')
                            Odds = soup.find_all('td', attrs=['class', 'Odds'])[m].contents[1]
                            if Odds.dt.string is not None:
                                dst['odds'] = Odds.dt.string.strip('倍')
                                dst['popu'] = soup.find_all('td', attrs=['class', 'Odds'])[0].contents[1].dd.string.strip('人気')
                        except:
                            break
                        dst.name = str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l) + numStr(m)
                    
                        df = df.append(dst)
df.to_csv('keiba' + str(year) + '.csv', encoding='shift-jis')

In [322]:
print(i, j, k, l, m, year)

3 3 5 12 13 2008


In [321]:
len(detailL.contents[0].split()[1].split('('))>=2

False

In [203]:
url = Base + str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l)
print(url)
time.sleep(1)
html = requests.get(url)
html.encoding = 'EUC-JP'
soup = BeautifulSoup(html.text, 'html.parser')

CommonYear = year
CommonDate = soup.find_all('div', attrs={'class', 'Change_Btn Day'})[0].string.strip()
CommonField= soup.find_all('div', attrs={'class', 'Change_Btn Course'})[0].string.strip()
CommonRace = soup.find_all('div', attrs={'Race_Num'})[0].span.string
CommonRname= soup.find_all('dt', attrs={'class', 'Race_Name'})[0].string.strip()
CommonCourse= soup.find_all('dd', attrs={'Race_Data'})[0].span.string
CommonHcount= soup.find_all('dd', attrs={'class', 'Race_Data'})[0].contents[3].split()[1]
m = 10
detailR = soup.find_all('span', attrs=['class', 'Detail_Right'])[m].contents
#print(detailR[0].strip())
#print(detailR[2].strip().replace('(', '').replace(')', ''))

http://race.sp.netkeiba.com/?pid=race_result&race_id=200801010102


In [82]:
# test
year = 2008
i = 1
j = 1
k = 1
l = 1
url = Base + str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l)
html = requests.get(url)
html.encoding = 'EUC-JP'

In [83]:
soup = BeautifulSoup(html.text, 'html.parser')

In [84]:
url

'http://race.sp.netkeiba.com/?pid=race_result&race_id=200801010101'

In [113]:
dst_tst = pd.Series(index = df_col)
#共通
dst_tst['year'] = year
dst_tst['date'] = soup.find_all('div', attrs={'class', 'Change_Btn Day'})[0].string.strip()
dst_tst['field']= soup.find_all('div', attrs={'class', 'Change_Btn Course'})[0].string.strip()
dst_tst['race'] = soup.find_all('div', attrs={'Race_Num'})[0].span.string
dst_tst['race_name'] = soup.find_all('dt', attrs={'class', 'Race_Name'})[0].string.strip()
dst_tst['course'] = soup.find_all('dd', attrs={'Race_Data'})[0].span.string
dst_tst['head_count'] = soup.find_all('dd', attrs={'class', 'Race_Data'})[0].contents[3].split()[1]
dst_tst

year             2008
date          8/16(土)
field              札幌
race               1R
race_name       ２歳未勝利
course         芝1500m
head_count        14頭
rank              NaN
horse_name        NaN
gender            NaN
age               NaN
trainerA          NaN
trainerB          NaN
weight            NaN
c_weight          NaN
jackie            NaN
j_weight          NaN
odds              NaN
popu              NaN
dtype: object

In [86]:
df_col

['year',
 'date',
 'field',
 'race',
 'race_name',
 'course',
 'head_count',
 'rank',
 'horse_name',
 'gender',
 'age',
 'trainerA',
 'trainerB',
 'weight',
 'c_weight',
 'jackie',
 'j_weight',
 'odds',
 'popu']

In [149]:
#馬別
dst_tst['rank'] = soup.find_all('div', attrs='Rank')[0].contents[0]
dst_tst['horse_name'] = soup.find_all('dt', attrs=['class', 'Horse_Name'])[0].a.string
detail = soup.find_all('span', attrs=['class', 'Detail_Left'])[0]
dst_tst['gender'] = list(detail.contents[0].split()[0])[0]
dst_tst['age'] = list(detail.contents[0].split()[0])[1]
dst_tst['trainerA'] = detail.span.string.split('･')[0]
dst_tst['trainerB'] = detail.span.string.split('･')[1]
dst_tst['weight'] = detail.contents[0].split()[1].split('(')[0]
dst_tst['c_weight'] = detail.contents[0].split()[1].split('(')[1].strip(')')
dst_tst['jackie'] = soup.find_all('span', attrs=['class', 'Detail_Right'])[0].contents[0].strip()
dst_tst['j_weight'] = soup.find_all('span', attrs=['class', 'Detail_Right'])[0].contents[2].strip().replace('(', '').replace(')', '')
dst_tst['odds'] = soup.find_all('td', attrs=['class', 'Odds'])[0].contents[1].dt.string.strip('倍')
dst_tst['popu'] = soup.find_all('td', attrs=['class', 'Odds'])[0].contents[1].dd.string.strip('人気')
dst_tst.name = 'new'

df.append(dst_tst)

,age,c_weight,course,date,field,gender,head_count,horse_name,j_weight,jackie,odds,popu,race,race_name,rank,trainerA,trainerB,weight,year
new,2,0,芝1500m,8/16(土),札幌,牝,14頭,ディアジーナ,54.0,四位,4.3,2,1R,２歳未勝利,1,美浦,田村,468,2008.0


In [77]:
p = soup.find_all('div', attrs={'class', 'Result_Guide'} )
if soup.find_all('div', attrs={'class', 'Result_Guide'})!=[]:
    pass
else:
    dst = pd.Series(index=df_col)
    dst['']

True


[<div class="Result_Guide">
 <p>30秒速報の情報はプレミアムサービス<br/>登録でご覧になれます。</p>
 <a class="mb20" href="https://regist.sp.netkeiba.com/">登録済みの方はこちらからログイン</a>
 <div class="MoreLinkBtn Detail"><a data-theme="01003" href="https://regist.sp.netkeiba.com/?pid=premium&amp;service=p43&amp;return_url=http%3A%2F%2Frace.sp.netkeiba.com%2F%3Fpid%3Drace_result%26race_id%3D200801011001" id="a_monthly_goods_link_01">30秒速報を見る</a></div>
 </div>]

In [32]:
html = requests.get(url)

In [33]:
html.encoding = 'EUC-JP'

In [34]:
html.text

'\n\n<!DOCTYPE html>\n<html>\n<head>\n<meta charset="EUC-JP">\n\n<!-- block=meta_tag_each_race (cp) -->\n<meta http-equiv="content-language" content="ja">\n<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=0.5, user-scalable=yes">\n\n<meta name="format-detection" content="telephone=no" />\n<meta name="description" content="2008年8月16日 札幌1R ２歳未勝利の結果・払戻です。JRA開催レースの出馬表や最新オッズ、レース結果速報、払戻情報をはじめ、競馬予想やデータ分析など予想に役立つ情報も満載です。" />\n<meta name="keywords" content="競馬,keiba,出馬表,オッズ,予想,レース結果,払戻し,結果速報,競馬予想,ネット競馬,netkeiba" />\n<meta name="thumbnail" content="http://www.netkeiba.com/style/netkeiba.ja/image/netkeiba.png" />\n<!-- ogp用 -->\n<meta property="og:site_name" content="netkeiba.com" />\n<meta property="og:type" content="article" />\n<meta property="og:title" content="２歳未勝利 結果・払戻 | 2008年8月16日 札幌1R レース情報(JRA) - netkeiba.com" />\n<meta property="og:url" content="http://race.netkeiba.com/?pid=race&id=c200801010101&mode=result" />\n<meta property="og

In [164]:
print(df)

               age c_weight  course     date field gender head_count  \
20080101010100   2        0  芝1500m  8/16(土)    札幌      牝        14頭   
20080101010101   2        0  芝1500m  8/16(土)    札幌      牡        14頭   
20080101010102   2       +2  芝1500m  8/16(土)    札幌      牡        14頭   
20080101010103   2       -4  芝1500m  8/16(土)    札幌      牡        14頭   
20080101010104   2      -20  芝1500m  8/16(土)    札幌      牝        14頭   
20080101010105   2       -4  芝1500m  8/16(土)    札幌      牡        14頭   
20080101010106   2      -10  芝1500m  8/16(土)    札幌      牡        14頭   
20080101010107   2       -6  芝1500m  8/16(土)    札幌      牡        14頭   
20080101010108   2       -8  芝1500m  8/16(土)    札幌      牡        14頭   
20080101010109   2       +2  芝1500m  8/16(土)    札幌      牡        14頭   
20080101010110   2        0  芝1500m  8/16(土)    札幌      牡        14頭   
20080101010111   2       -4  芝1500m  8/16(土)    札幌      牡        14頭   
20080101010112   2       -2  芝1500m  8/16(土)    札幌      牡       

[157 rows x 19 columns]
